<a href="https://colab.research.google.com/github/EeyoungSong/Capstone_Rendering_Algorithm/blob/main/%EC%A2%85%ED%95%A9%EC%84%A4%EA%B3%84_%EB%A0%8C%EB%8D%94%EB%A7%81_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np


def normalization(data):
    if type(data) == list:
        data = np.array(data)

    min_val = np.min(data[:])  # 두번째 열의 최솟값
    max_val = np.max(data[:])  # 두번째 열의 최댓값
    normalized_data = (data[:] - min_val) / (max_val - min_val)  # Min-Max 정규화

    # 0에서 255 사이의 값으로 변환
    scaled_data = normalized_data * 255

    # 변환된 값을 정수로 변환
    scaled_data = scaled_data.astype(int)

    # 변환된 값 출력
    # for time, value in zip(times[:], scaled_data):
    #     print(f"{time}\t{value}")

    # df_magnitude = pd.DataFrame({'Time (s)': times[:-1], 'trans_Magnitude': scaled_data})
    # # df_magnitude.to_csv('magnitude_trans_data.csv', index=False)

    return list(scaled_data)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
scaled_list = normalization(magnitude_list)

# 정규화 후 제곱 적용
def normalization_then_square(data):
    scaled_list = normalization(data)
    data_squared = [int((d**2/(255**2))*255) for d in scaled_list]
    return data_squared


# 제곱 적용 후 정규화
def square_then_normalization(data):
    data_squared = [int(d ** 2) for d in data]
    return normalization(data_squared)


# 세제곱 적용 후 정규화
def square_then_normalization_3(data):
    data_squared = [int(d ** 3) for d in data]
    return normalization(data_squared)


In [ ]:
# 높은 주파수로 갈 수록, 낮은 주파수로 갈 수록 강조
# 상대적
def high_low_emphasized():

    weight_list= []     # 가중치 리스트
    grad = 0.000001  # 이차함수 기울기
    min_freq = min(freq_list)
    max_freq = max(freq_list)
    x_vertex = (min_freq + max_freq) / 2 # 이차함수의 꼭짓점 x 좌표
    y_vertex = 0     # 이차함수의 꼭짓점 y 좌표

    for freq in freq_list:
        weight = grad*(freq - x_vertex)**2 + y_vertex
        weight_list.append(weight)


    # 가중치를 amplitude에 적용
    final_result = [mag if mag == 0 or int(mag+wei) > 255  else int(mag+wei) for mag,wei in zip(normalization(magnitude_list), weight_list)]

    return final_result


# 절대적
def high_low_emphasized_absolute(mag_list, freq_list):

    weight_list= []     # 가중치 리스트
    grad = 0.00005  # 이차함수 기울기
    x_vertex = 1000 # 이차함수의 꼭짓점 x 좌표
    y_vertex = 0     # 이차함수의 꼭짓점 y 좌표

    for freq in freq_list:
        weight = grad*(freq - x_vertex)**2 + y_vertex
        weight_list.append(weight)


    # 가중치를 amplitude에 적용
    final_result = [mag if mag == 0 or int(mag+wei) > 255  else int(mag+wei) for mag,wei in zip(normalization(mag_list), weight_list)]

    return final_result



## 테스트 예시 코드

In [ ]:
import numpy as np
import librosa
import librosa.display
import pandas as pd

file_path1 = "/content/drive/MyDrive/24-1/종합설계/오디오파일/총효과음/기관총 두둗.wav"
file_path2 = "/content/drive/MyDrive/24-1/종합설계/오디오파일/총효과음/후보군/탕.wav"
file_path3 = "/content/drive/MyDrive/24-1/종합설계/오디오파일/총효과음/피유웅(가볍게).wav"
file_path4 = "/content/drive/MyDrive/24-1/종합설계/오디오파일/데미지효과음/억.wav"
file_path5 = "/content/drive/MyDrive/24-1/종합설계/오디오파일/데미지효과음/으아아아앍.wav"
file_path6 = "/content/drive/MyDrive/24-1/종합설계/오디오파일/데미지효과음/으악.wav"

file_paths = [file_path1, file_path2, file_path3, file_path4, file_path5, file_path6]
file_names = ["기관총_v", "소총_v", "권총_v", "damage_small_v", "damage_big_v", "damage_middle_v"]


def get_stft(file_path):
    # 오디오 파일 로드
    sig, sr = librosa.load(file_path, sr=22050)
    a = len(sig)
    total_time = librosa.get_duration(y=sig,sr=sr)
    rms = librosa.feature.rms(y=sig)


    # STFT 파라미터 설정
    # n_fft = 2048 # FFT 크기
    # n_fft = 1024
    n_fft = 512

    hop_length = 256



    # STFT 계산
    stft = librosa.stft(sig, n_fft=n_fft, hop_length=hop_length)

    # 주파수 및 시간 계산
    freqs = librosa.fft_frequencies(sr=sr, n_fft=n_fft)


    times = np.linspace(0, total_time, stft.shape[1])


    # 주파수와 크기를 저장할 리스트 초기화
    freq_list = []
    magnitude_list = []

    # 각 구간의 주파수와 크기 저장
    for i, (start, end) in enumerate(zip(times[:-1], times[1:])):
        stft_segment = stft[:, i]  # 현재 구간의 STFT 결과
        magnitude = np.max(np.abs(stft_segment))  # 크기 계산
        max_magnitude_idx = np.argmax(np.abs(stft_segment)) # max값 인덱스

        freq = freqs[max_magnitude_idx]  # 주파수 계산 (가장 큰 진폭 크기를 가진 주파수)
        freq_list.append(freq)
        magnitude_list.append(np.max(magnitude))
    return [freq_list, magnitude_list]


In [ ]:
# 전체 실행 코드

for i in range(len(file_paths)):
    freq_list, mag_list = get_stft(file_paths[i])
    freq_list = freq_list[:250]
    mag_list = mag_list[:250]
    l = len(freq_list)

    simple = [l] + square_then_normalization(mag_list)
    high_low_emphasized_arr = [l] + high_low_emphasized_absolute(mag_list, freq_list)
    squared_high_low_emphasized_arr = [l] + square_then_normalization(high_low_emphasized_absolute(mag_list, freq_list))
    print(file_names[i])
    print(simple)
    print(high_low_emphasized_arr)
    print(squared_high_low_emphasized_arr)
    print()
    df_simple = pd.DataFrame(simple)
    df_simple.to_csv(f"/content/drive/MyDrive/24-1/csv_output/{file_names[i]}1.csv", index=False, header=False)
    df_high_low_emphasized = pd.DataFrame(high_low_emphasized_arr)
    df_high_low_emphasized.to_csv(f"/content/drive/MyDrive/24-1/csv_output/{file_names[i]}2.csv", index=False, header=False)
    df_squared_high_low_emphasized = pd.DataFrame(squared_high_low_emphasized_arr)
    df_squared_high_low_emphasized.to_csv(f"/content/drive/MyDrive/24-1/csv_output/{file_names[i]}3.csv", index=False, header=False)
    print()




기관총_v
[164, 0, 29, 88, 81, 112, 7, 5, 5, 48, 27, 147, 106, 3, 0, 17, 105, 203, 180, 15, 6, 9, 77, 37, 124, 142, 8, 2, 8, 94, 113, 178, 30, 4, 1, 57, 54, 146, 90, 8, 3, 14, 61, 40, 235, 124, 8, 4, 18, 83, 33, 124, 124, 4, 1, 12, 65, 60, 255, 44, 15, 2, 16, 77, 98, 154, 9, 5, 4, 3, 104, 42, 200, 53, 3, 7, 25, 79, 155, 166, 14, 4, 20, 53, 46, 148, 120, 8, 4, 14, 39, 63, 130, 136, 6, 2, 16, 90, 82, 231, 37, 13, 5, 10, 85, 18, 172, 86, 5, 1, 1, 92, 57, 131, 82, 4, 4, 24, 59, 170, 187, 10, 2, 7, 42, 43, 171, 129, 8, 2, 3, 51, 43, 114, 113, 15, 3, 6, 1, 2, 2, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[164, 0, 109, 190, 193, 213, 86, 81, 84, 146, 123, 230, 213, 56, 15, 68, 204, 227, 213, 106, 90, 98, 176, 141, 195, 240, 91, 57, 49, 195, 219, 212, 137, 81, 20, 157, 162, 243, 200, 90, 72, 109, 147, 145, 245, 227, 90, 77, 117, 182, 137, 227, 227, 52, 41, 105, 169, 172, 255, 155, 112, 24, 86, 180, 207, 243, 91, 84, 81, 51, 203, 152, 225, 165, 49, 93, 113, 182, 248, 25